In [12]:
# Importa pandas e funções do PySpark
import pandas as pd
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [14]:
# Inicializa a SparkSession
spark = SparkSession.builder.appName("InicializarDynamo").getOrCreate()

In [15]:
# Lê o CSV 
df = spark.read.csv('../data/amostragem.csv', header=True, inferSchema=True, sep=';')

In [16]:
# Substitui status de acorco com a descrição e remove o usuário diferente de Jeferson
df = df.withColumn(
    "status",
    when(col("Status Descrição") == "Concluído", "done")
    .when(col("Status Descrição") == "A Fazer", "todo")
    .otherwise(None)
)

In [17]:
# Mantém apenas linhas com status definido (done, todo)
df = df.filter(col("status").isNotNull())

In [18]:
# Mantém apenas linhas onde o usuário é 'Jeferson Klau'
df = df.filter(col("Usuário") == "Jeferson Klau")

In [19]:
# Remove a coluna Status Descrição
df = df.drop("Status Descrição")

# Remove a coluna Tipo da Tarefa ID
df = df.drop("Tipo da Tarefa ID")

In [20]:
# Renomeia colunas 
df = df.withColumnRenamed("Nome da Tarefa", "name") \
       .withColumnRenamed("Tipo da Tarefa", "task_type") \
       .withColumnRenamed("Data de Criação", "date") \
       .withColumnRenamed("Status", "status") 

In [21]:
# Visualiza os dados
df.show(15, truncate=False)

+---------------------------------------+------------------+-------------------+-------------------+------+-------------+--------------------------------+
|name                                   |task_type         |date               |Data de Conclusão  |status|Usuário      |ID do Usuário                   |
+---------------------------------------+------------------+-------------------+-------------------+------+-------------+--------------------------------+
|Cum iure exercitationem laboriosam     |Tarefa a Ser Feita|2024-07-18 22:51:51|2025-02-12 22:51:51|done  |Jeferson Klau|b4853fc1f03a3a4cec530a98a94d89ad|
|Asperiores iusto                       |Tarefa a Ser Feita|2025-05-29 09:07:49|NULL               |todo  |Jeferson Klau|b4853fc1f03a3a4cec530a98a94d89ad|
|Asperiores pariatur voluptatibus magnam|Tarefa a Ser Feita|2023-09-12 19:56:45|NULL               |todo  |Jeferson Klau|b4853fc1f03a3a4cec530a98a94d89ad|
|Comprar ipsa                           |Item de Compra    |2024-11-27